In [1]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib as mpl

width, height = 10, 5
mpl.rcParams['figure.figsize'] = [width, height]


In [2]:
from sklearn import svm
from sklearn import pipeline
from sklearn import preprocessing
from sklearn import model_selection

from utility_fns import payout_from_bid
from utility_fns import form_last_n_games
from utility_fns import make_train_val_test


In [3]:
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.compose import ColumnTransformer

from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline

from sklearn.metrics import precision_recall_fscore_support

from sklearn.ensemble import BaggingClassifier

from sklearn.tree import DecisionTreeClassifier


In [4]:
import xgboost as xgb
from catboost import CatBoostClassifier, Pool, metrics, cv


In [5]:
from catboost.utils import get_gpu_device_count

use_gpu = False
if get_gpu_device_count():
    use_gpu = True


In [6]:
# Global notebook params (speed processing of stuff we want only once)
use_years_as_weights = True

do_manual_svc = True
do_manual_bagging = True
do_sklearn_grid_tree = True
do_xgboost = True
do_catboost = True


In [7]:
basedir = os.path.dirname(os.getcwd())
data = os.path.join(basedir, 'data', 'derived', 'cleaned_matches.csv')
defense = os.path.join(basedir, 'data', 'derived', 'stats_categories', 'DEFENSE_stats.csv')
snaps = os.path.join(basedir, 'data', 'derived', 'stats_categories', 'SNAP_COUNTS_stats.csv')

cleaned_matches = pd.read_csv(data)
# defense_stats = pd.read_csv(defense, index_col=[0,1,2])
# snaps_stats = pd.read_csv(snaps, index_col=[0,1,2])

In [8]:
teams_and_codes = cleaned_matches.groupby(['Team', 'Team Code']).count().index.to_series().reset_index(level=1).drop(columns=[0])
opponents_and_codes = cleaned_matches.groupby(['Opponent', 'Opponent Code']).count().index.to_series().reset_index(level=1).drop(columns=[0])
teams_and_codes.columns = ['Code']
opponents_and_codes.columns = ['Code']

In [9]:
cleaned_matches.Date = pd.to_datetime(cleaned_matches.Date)
cleaned_matches.sort_values(by='Date', inplace=True, ascending=True)
cleaned_matches.drop(cleaned_matches.index[cleaned_matches.Date.dt.year >= 2008], inplace=True)
cleaned_matches.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 24884 entries, 0 to 24866
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Day             24884 non-null  object        
 1   Date            24884 non-null  datetime64[ns]
 2   Home/Away       24884 non-null  object        
 3   Opponent        24884 non-null  object        
 4   Score           24884 non-null  object        
 5   W/L/T           24884 non-null  object        
 6   Overtime        686 non-null    object        
 7   Location        24884 non-null  object        
 8   Venue           24883 non-null  object        
 9   Attendance      24568 non-null  float64       
 10  Notes           947 non-null    object        
 11  Opponent Code   24884 non-null  object        
 12  Team            24884 non-null  object        
 13  Team Code       24884 non-null  object        
 14  Team Score      24711 non-null  float64       
 15  Op

In [10]:
cleaned_matches['Pct Team Score'] = cleaned_matches['Team Score'] / \
    (cleaned_matches['Team Score'] + cleaned_matches['Opponent Score'])
cleaned_matches['Pct Opponent Score'] = cleaned_matches['Opponent Score'] / \
    (cleaned_matches['Team Score'] + cleaned_matches['Opponent Score'])

oldest_match_year = np.min(cleaned_matches['Date'].dt.year)

cleaned_matches['Weight'] = (cleaned_matches['Date'].dt.year - oldest_match_year
                             ) if use_years_as_weights else np.ones(cleaned_matches['Date'].shape)


## Train to classify based on the last-n-games a team played

In [22]:
x_cols = [
    'Team Code',
    'Opponent Code',
    'Home/Away'
]
lookup_x_cols = [
    'Team Score',
    'Opponent Score',
    'Score Diff',
    'Pct Team Score',
    'Pct Opponent Score',
    'Home/Away'
]
key_x_cols = [
    'Team Code',
    'Opponent Code'
]
weight_cols = [
    'Weight'
]
y_cols = [
    'Class'
]
date_cols_by_key = [['Date', 'Team Code_p_Date',
                     'Team Code_pp_Date',
                     'Team Code_ppp_Date',
                     'Team Code_pppp_Date',
                     'Team Code_ppppp_Date',
                     ],
                    ['Date', 'Opponent Code_p_Date',
                     'Opponent Code_pp_Date',
                     'Opponent Code_ppp_Date',
                     'Opponent Code_pppp_Date',
                     'Opponent Code_ppppp_Date',
                     ]]

filter_col = 'Date'

played_matches = cleaned_matches.dropna(
    axis=0, how='any', subset=x_cols + y_cols + lookup_x_cols).copy()

played_matches['Class'].where(
    played_matches['Class'] > 0, other=0, inplace=True)


played_matches.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 24579 entries, 0 to 24866
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Day                 24579 non-null  object        
 1   Date                24579 non-null  datetime64[ns]
 2   Home/Away           24579 non-null  object        
 3   Opponent            24579 non-null  object        
 4   Score               24579 non-null  object        
 5   W/L/T               24579 non-null  object        
 6   Overtime            686 non-null    object        
 7   Location            24579 non-null  object        
 8   Venue               24578 non-null  object        
 9   Attendance          24281 non-null  float64       
 10  Notes               929 non-null    object        
 11  Opponent Code       24579 non-null  object        
 12  Team                24579 non-null  object        
 13  Team Code           24579 non-null  object    

In [12]:
def dataset_transformer():
    return ColumnTransformer([('scaler', StandardScaler(), numeric_columns), ('one-hot', OneHotEncoder(handle_unknown='ignore'), categorical_columns)])


In [ ]:
last_n_matches, new_columns = form_last_n_games(
    played_matches, 5, lookup_x_cols, key_x_cols, filter_col)

for date_cols in date_cols_by_key:
  last_n_matches[date_cols] = last_n_matches[date_cols].drop(last_n_matches.index[last_n_matches[date_cols].diff(axis=1)[
      date_cols[1]].dt.days == 0]).diff(axis=1)
  last_n_matches.drop(columns=date_cols[0], inplace=True)

last_n_matches.head()

In [ ]:
X = pd.concat([last_n_matches[x_cols], last_n_matches[new_columns]], axis=1)
y = last_n_matches[y_cols]
weights = played_matches[weight_cols].loc[last_n_matches.index,:]

numeric_columns = X.columns[X.columns.str.contains('Score')]
categorical_columns = X.columns[~X.columns.str.contains('Score')]
categorical_columns_indices = np.where(~X.columns.str.contains('Score'))[0]

print(categorical_columns)
print(categorical_columns_indices)
print(numeric_columns)
print(weights)


In [ ]:
X.head()


In [ ]:
X.info()


In [ ]:
# X.corr()


In [ ]:
y.head()


In [ ]:
last_1_pct = len(X) - len(X)//20
X_chrono_test = X.iloc[last_1_pct:]
y_chrono_test = y.iloc[last_1_pct:]
weight_chrono_test = weights.loc[X_chrono_test.index,:]

X_train, X_val, X_test, y_train, y_val, y_test = make_train_val_test(
    X, y, test_pct=0.2, val_pct=0.2, random_state=0
)

weight_train = weights.loc[X_train.index,:]
weight_val = weights.loc[X_val.index,:]
weight_test = weights.loc[X_test.index,:]

print(X.shape, y.shape, weights.shape)
print(X_train.shape, y_train.shape, weight_train.shape)
print(X_val.shape, y_val.shape, weight_val.shape)
print(X_test.shape, y_test.shape, weight_test.shape)
print(X_chrono_test.shape, y_chrono_test.shape, weight_chrono_test.shape)


In [ ]:
X_cv = pd.concat([X_train, X_val])
y_cv = pd.concat([y_train, y_val])
weight_cv = pd.concat([weight_train, weight_val])

In [ ]:
def manual_grid_search_svc(do=True):
    if not do:
        return None
    best_model = None
    for kernel in ['linear', 'poly', 'rbf', 'sigmoid']:
        train_scores = []
        val_scores = []
        val_space = np.geomspace(start=0.01, stop=5, num=3)
        for C in val_space:
            print(f"Evaluating {kernel} kernel for C={C}")
            model = make_pipeline(dataset_transformer(), svm.SVC(
                C=C, max_iter=10_000, kernel=kernel))

            model.fit(X_train, y_train.values.ravel(), svc__sample_weight=weight_train.values.ravel())
            train_scores.append(model.score(X_train, y_train.values.ravel(), weight_train.values.ravel()))
            val_scores.append(model.score(X_val, y_val.values.ravel(), weight_val.values.ravel()))

            if best_model is None or val_scores[-1] == np.max(val_scores):
                best_model = model

        plt.title(f'Best C for {kernel}: {val_space[np.argmax(val_scores)]}')
        plt.plot(val_space, train_scores, label='Train score')
        plt.plot(val_space, val_scores, label='Val score')
        # plt.ylim(0, 1.1)
        plt.xscale('log')
        plt.legend()
        plt.show()
    return best_model


best_model = manual_grid_search_svc(do_manual_svc)


In [ ]:
if best_model is not None:
    y_pred = best_model.predict(X_test)
    precision, recall, fbeta_score, _ = precision_recall_fscore_support(
        y_test, y_pred, average='binary')
    print(precision, recall, fbeta_score)


In [ ]:
def manual_grid_bagging(do=True):
    if not do:
        return [], [], 0, 0
    best_config = []
    best_score = 0
    best_fit_config = []
    best_fit_score = 0
    for estimator in [DecisionTreeClassifier(max_leaf_nodes=3), DecisionTreeClassifier(max_leaf_nodes=5), DecisionTreeClassifier(max_leaf_nodes=10), DecisionTreeClassifier(max_leaf_nodes=50), DecisionTreeClassifier()]:
        for nestimators in [10, 31, 62, 93, 124, 200, 300, 400, 500]:
            for nsamples in [0.01, 0.02, 0.05, 0.1, 0.2, 0.4, 0.6]:
                clf = make_pipeline(dataset_transformer(), BaggingClassifier(
                    base_estimator=estimator, n_estimators=nestimators, max_samples=nsamples))
                clf.fit(X_train, y_train.values.ravel(
                ), baggingclassifier__sample_weight=weight_train.values.ravel())

                score = clf.score(X_val, y_val.values.ravel(),
                                  weight_val.values.ravel())
                if(score > best_score):
                    best_config = [estimator, nestimators, nsamples]
                    best_score = score

                score = clf.score(X_train, y_train.values.ravel(),
                                  weight_train.values.ravel())
                if(score > best_fit_score):
                    best_fit_config = [estimator, nestimators, nsamples]
                    best_fit_score = score

                print(end='.')
            print(end='*')
        print(end='#')
    return best_config, best_fit_config, best_score, best_fit_score


best_config, best_fit_config, best_score, best_fit_score = manual_grid_bagging(
    do_manual_bagging)


In [ ]:
print(f"Best validation: {best_score}, {best_config}")
print(f"Best training: {best_fit_score}, {best_fit_config}")


In [ ]:
if do_manual_bagging:
    bagging_model = make_pipeline(dataset_transformer(), BaggingClassifier(
        base_estimator=best_config[0], n_estimators=best_config[1], max_samples=best_config[2]))
    bagging_model.fit(X_train, y_train.values.ravel(
    ), baggingclassifier__sample_weight=weight_train.values.ravel())


In [ ]:
model = {
    'model': BaggingClassifier,
    'kwargs': {'base_estimator': DecisionTreeClassifier()},
    'cross_vals': 3,
    'param_grid': {
        'bagging_clf__base_estimator__max_leaf_nodes': np.linspace(2, 100, num=4, dtype=int),
        'bagging_clf__max_samples': np.linspace(0.01, 0.8, 4),
        'bagging_clf__n_estimators': np.geomspace(10, 100, num=3, dtype=int)
    }
}

pipe = Pipeline([('column_transformer', dataset_transformer()),
                 ('bagging_clf', model['model'](**model['kwargs']))])

print(model)

grid_search_bag_tree = model_selection.GridSearchCV(
    pipe, param_grid=model['param_grid'], cv=model['cross_vals'], n_jobs=5, refit=True, verbose=3)


In [ ]:
if do_sklearn_grid_tree:
    grid_search_bag_tree.fit(X_cv, y_cv.values.ravel(), **{
                             'bagging_clf__sample_weight': weight_cv.values.ravel()})
                             


In [ ]:
if do_sklearn_grid_tree:
    # print(grid_search_bag_tree.cv_results_.keys())

    x = grid_search_bag_tree.cv_results_['rank_test_score']
    plt.bar(x, grid_search_bag_tree.cv_results_['mean_fit_time'])
    plt.title('Fit time vs Model Rank')
    plt.xlabel('Model rank in grid search')
    plt.ylabel('Fit Time (s)')
    plt.show()

    plt.errorbar(x, grid_search_bag_tree.cv_results_['mean_test_score'], grid_search_bag_tree.cv_results_['std_test_score'], fmt='d', markersize=5, label='Mean & std test score')
    plt.scatter(x, grid_search_bag_tree.cv_results_['split0_test_score'], alpha=0.2, label='Split 0 test score')
    plt.scatter(x, grid_search_bag_tree.cv_results_['split1_test_score'], alpha=0.2, label='Split 1 test score')
    plt.scatter(x, grid_search_bag_tree.cv_results_['split2_test_score'], alpha=0.2, label='Split 2 test score')
    plt.scatter(x, grid_search_bag_tree.cv_results_['split3_test_score'], alpha=0.2, label='Split 3 test score')
    plt.scatter(x, grid_search_bag_tree.cv_results_['split4_test_score'], alpha=0.2, label='Split 4 test score')
    plt.title('Test Score')
    plt.xlabel('Model rank in grid search')
    plt.ylabel('Model score on test set (%)')
    plt.legend()
    plt.show()

In [ ]:
if do_sklearn_grid_tree:
  print(
      f"Test score={grid_search_bag_tree.best_estimator_.score(X_test,y_test.values.ravel(), weight_test.values.ravel())}")
  print(
      f"Chronological test score={grid_search_bag_tree.best_estimator_.score(X_chrono_test, y_chrono_test.values.ravel(), weight_chrono_test.values.ravel())}")


In [ ]:
if do_sklearn_grid_tree:
    confidences = grid_search_bag_tree.best_estimator_.predict_proba(X_test)
    confidences = (2*np.max(confidences, axis=1)-1) * (2*np.argmax(confidences, axis=1) - 1) * (2 * y_test.values.ravel() - 1)
    wrong_preds = confidences[confidences < 0]
    right_preds = confidences[confidences > 0]
    print(f"Scores for last 5% of games={grid_search_bag_tree.best_estimator_.score(X_chrono_test, y_chrono_test)}")
    plt.title(f'Model Confidences for Right and Wrong Predictions, test accuracy={grid_search_bag_tree.best_estimator_.score(X_test, y_test.values.ravel())}')
    plt.hist(right_preds, bins=25, label='Right Predictions')
    plt.hist(wrong_preds, bins=25, label='Wrong Predictions')
    plt.legend()
    plt.xlabel('Model Confidence')
    plt.ylabel('Count')
    plt.show()


In [ ]:
clf = make_pipeline(dataset_transformer(), DecisionTreeClassifier())
clf.fit(X_train, y_train.values.ravel(), decisiontreeclassifier__sample_weight=weight_train.values.ravel())
clf.score(X_val, y_val.values.ravel(), weight_val.values.ravel())


In [ ]:
confidences = clf.predict_proba(X_test)
confidences = (2*np.max(confidences, axis=1)-1) * (2*np.argmax(confidences, axis=1) - 1) * (2 * y_test.values.ravel() - 1)
wrong_preds = confidences[confidences < 0]
right_preds = confidences[confidences > 0]
print(f"Scores for last 5% of games={clf.score(X_chrono_test, y_chrono_test.values.ravel())}")
plt.title(f'Model Confidences for Right and Wrong Predictions, test accuracy={clf.score(X_test, y_test.values.ravel())}')
plt.hist(right_preds, bins=25, label='Right Predictions')
plt.hist(wrong_preds, bins=25, label='Wrong Predictions')
plt.legend()
plt.show()


In [ ]:
if do_manual_svc:
    Cs = np.geomspace(0.01, 15, num=75)
    scores = []
    for c in Cs:
        model = make_pipeline(dataset_transformer(), svm.LinearSVC(
            dual=False, C=c, max_iter=100_000))
        scores.append(cross_val_score(
            model, X_cv, y_cv.values.ravel(), cv=5, fit_params={'linearsvc__sample_weight': weight_cv.values.ravel()}, n_jobs=-1))
        print(end='.')


In [ ]:
if do_manual_svc:
    x = Cs
    plt.errorbar(x, np.mean(scores, axis=1), np.std(scores, axis=1),
                 label='Mean & Std Dev. of CV Score', linewidth=1)
    plt.plot(x, np.sort(scores, axis=1), linewidth=0.2)
    plt.legend()
    plt.xscale('log')
    plt.xlabel('C')
    plt.ylabel('Accuracy')
    plt.title('Cross-Validation Score')
    plt.show()


In [ ]:
if do_manual_svc:
    print('Maximum mean:', np.max(np.mean(scores, axis=1)))
    print('Corresponding std deviation:', np.std(
        scores, axis=1)[np.argmax(np.mean(scores, axis=1))])
    print('C:', Cs[np.argmax(np.mean(scores, axis=1))])


In [ ]:
if do_manual_svc:
    print('Minimum std deviation:', np.min(np.std(scores, axis=1)))
    print('Corresponding mean:', np.mean(scores, axis=1)
          [np.argmin(np.std(scores, axis=1))])
    print('C:', Cs[np.argmin(np.std(scores, axis=1))])


In [ ]:
if do_manual_svc:
    best_mean_cs = np.flip(np.argsort(np.mean(scores, axis=1)))
    best_std_cs = np.argsort(np.std(scores, axis=1))


In [ ]:
if do_manual_svc:
    # Do some random perturbation of the Cs
    scores = []

    Cs_to_examine = np.unique(np.concatenate(
        (Cs[best_mean_cs[:5]], Cs[best_std_cs[:5]])))
    Cs_to_examine = np.unique(np.concatenate([Cs_to_examine, np.abs(
        Cs_to_examine + np.random.normal(scale=0.2, size=len(Cs_to_examine)))]))

    for c in Cs_to_examine:
        model = make_pipeline(dataset_transformer(), svm.LinearSVC(
            dual=False, C=c, max_iter=100_000))

        scores.append(cross_val_score(
            model, X_cv, y_cv.values.ravel(), cv=5, n_jobs=-1, fit_params={'linearsvc__sample_weight': weight_cv.values.ravel()}))

        print(end='.')


In [ ]:
if do_manual_svc:
    x = Cs_to_examine
    quantiles = [0, 0.1, 0.2, 0.3, 0.4, 0.6, 0.8, 1.0]
    plt.errorbar(x, np.mean(scores, axis=1), np.std(scores, axis=1),
                 label='Mean & Std Dev. of CV Score', linewidth=1)
    plt.plot(x, np.quantile(scores, quantiles, axis=1).T, linewidth=0.2)
    plt.legend()
    plt.xscale('log')
    plt.xlabel('C')
    plt.ylabel('Accuracy')
    plt.title('Cross-Validation Score')
    plt.show()


In [ ]:
if do_manual_svc:
    best_mean_idx = np.argmax(np.mean(scores, axis=1))

    print(
        f'Best C={Cs_to_examine[best_mean_idx]} with mean={np.mean(scores,axis=1)[best_mean_idx]}, std={np.std(scores,axis=1)[best_mean_idx]}')


In [ ]:
if do_manual_svc:
    # Train the best model.

    model = make_pipeline(dataset_transformer(),
                          svm.LinearSVC(dual=False, C=Cs_to_examine[best_mean_idx], max_iter=100_000))
    model.fit(X_cv, y_cv.values.ravel(),
              linearsvc__sample_weight=weight_cv.values.ravel())
    model.score(X_test, y_test.values.ravel())


In [ ]:
if do_manual_svc:
    coefs = model[1].coef_[0]
    plt.scatter(x=np.arange(len(coefs)), y=coefs)
    plt.show()


In [ ]:
if do_manual_svc:
    model.fit(X, y.values.ravel(),
              linearsvc__sample_weight=weights.values.ravel())
    confidences = model.decision_function(X)
    print(f"Scores for last games={model.score(X_chrono_test, y_chrono_test.values.ravel(), weight_chrono_test.values.ravel())}")
    print(f"Scores for test games={model.score(X_test, y_test.values.ravel(), weight_test.values.ravel())}")


In [ ]:
if do_manual_svc:
    plt.hist(confidences, bins=25)
    plt.show()


In [ ]:
if do_manual_svc:
    plt.hist(confidences * (2 * y.to_numpy().ravel() - 1), bins=25)
    plt.show()


In [ ]:
if do_manual_svc:
    np.min(confidences * (2 * y.to_numpy().ravel() - 1))


In [ ]:
if do_catboost:
    catboost_model = CatBoostClassifier(
        custom_loss=[metrics.Accuracy()],
        random_seed=0xdeadbeef,
        logging_level='Silent',
        task_type='GPU' if use_gpu else 'CPU'
    )

    catboost_model.fit(
        X_cv, y_cv,sample_weight=weight_cv,
        cat_features=categorical_columns_indices,
        eval_set=(X_test, y_test),
        early_stopping_rounds=100,
        plot=True
    )


In [ ]:
if do_catboost:
    data_pool = Pool(X_cv, y_cv, cat_features=categorical_columns_indices, weight=weight_cv)
    cv_params = catboost_model.get_params()
    cv_params.update({
        'loss_function': metrics.Logloss()
    })
    cv_data = cv(
        data_pool,
        cv_params,
        plot=True,
        early_stopping_rounds=100
    )


In [ ]:
if do_catboost:
    print('Best validation accuracy score: {:.2f}±{:.2f} on step {}'.format(
        np.max(cv_data['test-Accuracy:use_weights=true-mean']),
        cv_data['test-Accuracy:use_weights=true-std'][np.argmax(cv_data['test-Accuracy:use_weights=true-mean'])],
        np.argmax(cv_data['test-Accuracy:use_weights=true-mean'])
    ))
    print()
    
    feature_importances = catboost_model.get_feature_importance(data_pool)
    feature_names = X_train.columns
    for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
        print('{}: {}'.format(name, score))


In [ ]:
if do_catboost:
    confidences = catboost_model.predict_proba(X_test)
    confidences = (2*np.max(confidences, axis=1)-1) * (2*np.argmax(confidences, axis=1) - 1) * (2 * y_test.values.ravel() - 1)
    wrong_preds = confidences[confidences < 0]
    right_preds = confidences[confidences > 0]
    print(f"Scores for last 5% of games={catboost_model.score(X_chrono_test, y_chrono_test)}")
    plt.title(f'Model Confidences for Right and Wrong Predictions, test accuracy={catboost_model.score(X_test, y_test.values.ravel())}')
    plt.hist(right_preds, bins=25, label='Right Predictions')
    plt.hist(wrong_preds, bins=25, label='Wrong Predictions')
    plt.legend()
    plt.show()


In [ ]:
if do_xgboost:
  xgboost_model = make_pipeline(dataset_transformer(),xgb.XGBClassifier(use_label_encoder=False, objective='binary:logistic', eval_metric='logloss'))
  xgboost_model.fit(X_cv, y_cv.values.ravel(), xgbclassifier__sample_weight=weight_cv.values.ravel())
  print(f'Train accuracy={xgboost_model.score(X_cv, y_cv.values.ravel())}')
  
  confidences = xgboost_model.predict_proba(X_test)
  confidences = (2*np.max(confidences, axis=1)-1) * (2*np.argmax(confidences, axis=1) - 1) * (2 * y_test.values.ravel() - 1)
  wrong_preds = confidences[confidences < 0]
  right_preds = confidences[confidences > 0]
  print(f"Scores for last 5% of games={xgboost_model.score(X_chrono_test, y_chrono_test.values.ravel(), weight_chrono_test.values.ravel())}")
  plt.title(f'Model Confidences for Right and Wrong Predictions, test accuracy={xgboost_model.score(X_test, y_test.values.ravel(), weight_test.values.ravel())}')
  plt.hist(right_preds, bins=25, label='Right Predictions')
  plt.hist(wrong_preds, bins=25, label='Wrong Predictions')
  plt.legend()
  plt.show()

In [12]:
odds_data = pd.read_csv(os.path.join(basedir, 'data', 'derived', 'odds_portal_clean.csv'))
odds_data = odds_data.loc[odds_data['Special Game'].str.contains('All Stars').isna(),:]

odds_data['Date'] = pd.to_datetime(odds_data['Date'])
odds_data['Home Score'] = pd.to_numeric(odds_data['Home Score'])
odds_data['Away Score'] = pd.to_numeric(odds_data['Away Score'].str.strip())

odds_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3414 entries, 0 to 4328
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Season        3414 non-null   object        
 1   Date          3414 non-null   datetime64[ns]
 2   Special Game  0 non-null      object        
 3   Time          3414 non-null   object        
 4   Home Team     3414 non-null   object        
 5   Away Team     3414 non-null   object        
 6   Home Score    3414 non-null   int64         
 7   Away Score    3414 non-null   int64         
 8   OT            3414 non-null   bool          
 9   Winner        3414 non-null   object        
 10  Home Odds     3414 non-null   object        
 11  Away Odds     3414 non-null   object        
 12  B's           3414 non-null   int64         
dtypes: bool(1), datetime64[ns](1), int64(3), object(8)
memory usage: 350.1+ KB


In [13]:
team_to_code = pd.concat([teams_and_codes, opponents_and_codes])


In [14]:
odds_data['Home Code'] = team_to_code.loc[odds_data['Home Team'].str.strip().to_list(), 'Code'].reset_index(drop=True)
odds_data['Away Code'] = team_to_code.loc[odds_data['Away Team'].str.strip().to_list(), 'Code'].reset_index(drop=True)
odds_data.head()

,Season,Date,Special Game,Time,Home Team,Away Team,Home Score,Away Score,OT,Winner,Home Odds,Away Odds,B's,Home Code,Away Code
0,2021/22,2021-11-16,NaN,1:15,San Francisco 49ers,Los Angeles Rams,31,10,False,Home,158,-182,13,sf,larm
1,2021/22,2021-11-15,NaN,1:20,Las Vegas Raiders,Kansas City Chiefs,14,41,False,Away,129,-149,13,sf,larm
2,2021/22,2021-11-14,NaN,21:25,Denver Broncos,Philadelphia Eagles,13,30,False,Away,-106,-110,13,lv,kc
3,2021/22,2021-11-14,NaN,21:25,Green Bay Packers,Seattle Seahawks,17,0,False,Home,-172,149,13,den,kc
4,2021/22,2021-11-14,NaN,21:05,Arizona Cardinals,Carolina Panthers,10,34,False,Away,-323,265,13,den,phi


In [15]:
pertinent_cols = [
    'Date',
    'Home Score',
    'Away Score',
    'Home Odds',
    'Away Odds',
    'Home Code',
    'Away Code'
]

odds_as_teams = odds_data.loc[:, pertinent_cols]
odds_as_opponents = odds_data.loc[:, pertinent_cols]

odds_as_teams.columns = [col.replace('Home', 'Team').replace(
    'Away', 'Opponent') for col in pertinent_cols]
odds_as_opponents.columns = [col.replace('Home', 'Opponent').replace(
    'Away', 'Team') for col in pertinent_cols]

odds_as_teams['Home/Away'] = 'H'
odds_as_opponents['Home/Away'] = 'A'

odds_as_matches = pd.concat([odds_as_teams, odds_as_opponents]).sort_values(
    ['Date', 'Team Code','Opponent Code'], ascending=True)

odds_as_matches['Weight'] = (odds_as_matches['Date'].dt.year - oldest_match_year
                             ) if use_years_as_weights else np.ones(odds_as_matches['Date'].shape)

odds_as_matches = odds_as_matches.reset_index(drop=True)

odds_as_matches.head()


,Date,Team Score,Opponent Score,Team Odds,Opponent Odds,Team Code,Opponent Code,Home/Away,Weight
0,2008-10-05,6,44,258,-303,bal,gb,A,88
1,2008-10-05,10,17,-263,225,bal,min,A,88
2,2008-10-05,13,10,-127,112,chib,pit,A,88
3,2008-10-05,0,34,371,-455,chib,pit,A,88
4,2008-10-05,34,7,-185,163,chic,ten,A,88


In [16]:
score_sum = odds_as_matches['Team Score'] + odds_as_matches['Opponent Score']
odds_as_matches['Pct Team Score'] = odds_as_matches['Team Score'] / score_sum
odds_as_matches['Pct Opponent Score'] = odds_as_matches['Opponent Score'] / score_sum

odds_as_matches['Score Diff'] = odds_as_matches['Team Score'] - \
    odds_as_matches['Opponent Score']

odds_as_matches['Class'] = pd.to_numeric(
    odds_as_matches['Team Score'] > odds_as_matches['Opponent Score']) * 1

odds_as_matches.head()


,Date,Team Score,Opponent Score,Team Odds,Opponent Odds,Team Code,Opponent Code,Home/Away,Weight,Pct Team Score,Pct Opponent Score,Score Diff,Class
0,2008-10-05,6,44,258,-303,bal,gb,A,88,0.120000,0.880000,-38,0
1,2008-10-05,10,17,-263,225,bal,min,A,88,0.370370,0.629630,-7,0
2,2008-10-05,13,10,-127,112,chib,pit,A,88,0.565217,0.434783,3,1
3,2008-10-05,0,34,371,-455,chib,pit,A,88,0.000000,1.000000,-34,0
4,2008-10-05,34,7,-185,163,chic,ten,A,88,0.829268,0.170732,27,1


In [17]:
last_n_matches, new_columns = form_last_n_games(
    odds_as_matches, 5, lookup_x_cols, key_x_cols, filter_col)

.................................................
 Done


In [18]:
for date_cols in date_cols_by_key:
  last_n_matches[date_cols] = last_n_matches[date_cols].drop(last_n_matches.index[last_n_matches[date_cols].diff(axis=1)[
      date_cols[1]].dt.days == 0]).diff(axis=1)
  last_n_matches.drop(columns=date_cols[0], inplace=True)
last_n_matches.head()

,Team Score,Opponent Score,Team Odds,Opponent Odds,Team Code,Opponent Code,Home/Away,Weight,Pct Team Score,Pct Opponent Score,...,Team Code_p_Date,Opponent Code_p_Date,Team Code_pp_Date,Opponent Code_pp_Date,Team Code_ppp_Date,Opponent Code_ppp_Date,Team Code_pppp_Date,Opponent Code_pppp_Date,Team Code_ppppp_Date,Opponent Code_ppppp_Date
202,48,28,-130,114,ari,bal,A,88,0.631579,0.368421,...,-2 days,2008-11-21,-5 days,2008-11-14,-21 days,2008-11-11,-7 days,2008-10-26,-5 days,2008-10-12
203,28,48,114,-130,bal,ari,H,88,0.368421,0.631579,...,-2 days,2008-11-21,-7 days,2008-11-16,-3 days,2008-10-26,-16 days,2008-10-19,-14 days,2008-10-14
244,20,13,164,-179,hou,jac,A,88,0.606061,0.393939,...,-7 days,2008-11-23,-13 days,2008-11-10,-1 days,2008-11-09,-14 days,2008-10-26,-7 days,2008-10-21
245,13,20,-179,164,jac,hou,H,88,0.393939,0.606061,...,-7 days,2008-11-23,-13 days,2008-11-10,-1 days,2008-11-09,-14 days,2008-10-26,-5 days,2008-10-19
260,14,34,197,-233,hou,jac,A,88,0.291667,0.708333,...,-1 days,2008-11-30,-7 days,2008-11-23,-13 days,2008-11-10,-1 days,2008-11-09,-14 days,2008-10-26


In [19]:
X_odds = pd.concat([last_n_matches[x_cols], last_n_matches[new_columns]], axis=1)
y_odds = last_n_matches[y_cols]

odds = last_n_matches.loc[:, ['Team Odds','Opponent Odds']]
odds['Team Odds'] = pd.to_numeric(odds['Team Odds'])
odds['Opponent Odds'] = pd.to_numeric(odds['Opponent Odds'])

odds_weights = odds_as_matches[weight_cols].loc[last_n_matches.index, :]

numeric_columns = X_odds.columns[X_odds.columns.str.contains('Score')]
categorical_columns = X_odds.columns[~X_odds.columns.str.contains('Score')]
categorical_columns_indices = np.where(~X_odds.columns.str.contains('Score'))[0]

print(categorical_columns)
print(categorical_columns_indices)
print(numeric_columns)
print(odds_weights)


Index(['Team Code', 'Opponent Code', 'Home/Away', 'Team Code_p_Home/Away',
       'Opponent Code_p_Home/Away', 'Team Code_pp_Home/Away',
       'Opponent Code_pp_Home/Away', 'Team Code_ppp_Home/Away',
       'Opponent Code_ppp_Home/Away', 'Team Code_pppp_Home/Away',
       'Opponent Code_pppp_Home/Away', 'Team Code_ppppp_Home/Away',
       'Opponent Code_ppppp_Home/Away', 'Team Code_p_Date',
       'Opponent Code_p_Date', 'Team Code_pp_Date', 'Opponent Code_pp_Date',
       'Team Code_ppp_Date', 'Opponent Code_ppp_Date', 'Team Code_pppp_Date',
       'Opponent Code_pppp_Date', 'Team Code_ppppp_Date',
       'Opponent Code_ppppp_Date'],
      dtype='object')
[ 0  1  2 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72]
Index(['Team Code_p_Team Score', 'Opponent Code_p_Team Score',
       'Team Code_pp_Team Score', 'Opponent Code_pp_Team Score',
       'Team Code_ppp_Team Score', 'Opponent Code_ppp_Team Score',
       'Team Code_pppp_Team Score', 'Opponent Code_pppp_Team Score',

In [20]:
def preds_to_confidences(preds):
    return np.array(2*np.max(preds, axis=1)-1) * (2*np.argmax(preds, axis=1) - 1).ravel()


def straight_bet_strategy(preds, odds, actual):
    confidences = preds_to_confidences(preds)
    odds_of_bet = np.where(
        confidences >= 0, odds['Team Odds'].values.ravel(), odds['Opponent Odds'].values.ravel())
    results = (confidences * (2*actual.values.ravel() - 1)) > 0

    bids = np.ones(results.shape)
    winnings = payout_from_bid(bids, odds_of_bet, results)

    return winnings, bids


def proportional_bet_strategy(preds, odds, actual):
    confidences = preds_to_confidences(preds)
    odds_of_bet = np.where(
        confidences >= 0, odds['Team Odds'].values.ravel(), odds['Opponent Odds'].values.ravel())
    results = (confidences * (2*actual.values.ravel() - 1)) > 0

    # Make integrated bid over confidences equal to straight, even though the distribution is not uniform
    bids = np.abs(confidences) + 0.5
    bids = bids * (len(bids) / np.sum(bids))  # Fixed amount of money to play with

    winnings = payout_from_bid(bids, odds_of_bet, results)

    return winnings, bids


def quadratic_proportional_bet_strategy(preds, odds, actual):
    confidences = preds_to_confidences(preds)
    odds_of_bet = np.where(
        confidences >= 0, odds['Team Odds'].values.ravel(), odds['Opponent Odds'].values.ravel())
    results = (confidences * (2*actual.values.ravel() - 1)) > 0

    # Make integrated bid over confidences equal to straight, even though the distribution is not uniform
    bids = np.square(np.abs(confidences)) + 2/3
    bids = bids * (len(bids) / np.sum(bids))  # Fixed amount of money to play with

    winnings = payout_from_bid(bids, odds_of_bet, results)

    return winnings, bids


def risky_bet_strategy(preds, odds, actual):
    confidences = preds_to_confidences(preds)
    odds_of_bet = np.where(
        confidences >= 0, odds['Team Odds'].values.ravel(), odds['Opponent Odds'].values.ravel())
    results = (confidences * (2*actual.values.ravel() - 1)) > 0

    # Make integrated bid over confidences equal to straight, even though the distribution is not uniform
    assumed_payout = payout_from_bid(
        np.ones(confidences.shape), odds_of_bet, np.ones(confidences.shape))
    bids = np.abs(confidences) * assumed_payout
    bids = bids * (len(bids) / np.sum(bids))  # Fixed amount of money to play with

    winnings = payout_from_bid(bids, odds_of_bet, results)

    return winnings, bids


def plot_bet_strategies(preds, odds, actual):
    straight_winnings, straight_bets = straight_bet_strategy(
        preds, odds, actual)
    proportional_winnings, proportional_bets = proportional_bet_strategy(
        preds, odds, actual)
    quadratic_winnings, quadratic_bets = quadratic_proportional_bet_strategy(
        preds, odds, actual)
    risky_winnings, risky_bets = risky_bet_strategy(preds, odds, actual)

    x = preds_to_confidences(preds)
    plt.scatter(x[straight_winnings != 0], straight_winnings[straight_winnings !=
                                                             0], label='Straight Bets Winnings', alpha=0.1, marker='.')
    plt.scatter(x[proportional_winnings != 0], proportional_winnings[proportional_winnings != 0],
                label='Proportional Bets Winnings', alpha=0.1, marker='.')
    plt.scatter(x[quadratic_winnings != 0], quadratic_winnings[quadratic_winnings !=
                                                               0], label='Quadratic Bets Winnings', alpha=0.1, marker='.')
    plt.scatter(x[risky_winnings != 0], risky_winnings[risky_winnings !=
                                                       0], label='Risky Bets Winnings', alpha=0.1, marker='.')

    straight_losses = -straight_bets[straight_winnings == 0]
    proportional_losses = -proportional_bets[proportional_winnings == 0]
    quadratic_losses = -quadratic_bets[quadratic_winnings == 0]
    risky_losses = -risky_bets[risky_winnings == 0]
    plt.scatter(x[straight_winnings == 0], straight_losses,
                label='Straight Bets Losses', alpha=0.1, marker='.')
    plt.scatter(x[proportional_winnings == 0], proportional_losses,
                label='Proportional Bets Losses', alpha=0.1, marker='.')
    plt.scatter(x[quadratic_winnings == 0], quadratic_losses,
                label='Quadratic Bets Losses', alpha=0.1, marker='.')
    plt.scatter(x[risky_winnings == 0], risky_losses,
                label='Risky Bets Losses', alpha=0.1, marker='.')

    plt.legend()
    plt.xlabel('Model Confidence')
    plt.ylabel('Payout')
    plt.title('Payout vs Losses for Three Strategies vs. Model Confidence')
    plt.show()

    net_straight = straight_winnings - straight_bets
    net_proportional = proportional_winnings - proportional_bets
    net_quadratic = quadratic_winnings - quadratic_bets
    net_risky = risky_winnings - risky_bets

    cols = ['Straight', 'Proportional', 'Quadratic', 'Risky']
    net_data = pd.DataFrame(data={'Straight': net_straight, 'Proportional': net_proportional,
                                  'Quadratic': net_quadratic, 'Risky': net_risky}, index=x).sort_index()
    binned = net_data.groupby([pd.cut(net_data.index, np.linspace(-1,1,21))])[
        cols].sum().reset_index()
    binned.plot.bar(x='index', y=cols)
    plt.title('Net Payout for Three Strategies vs Model Confidence')
    plt.ylabel('Payout')
    plt.show()

    print(net_data.sum(axis=0))

    binned.drop([binned.index[0], binned.index[-1]]).plot.bar(x='index',y=cols)
    plt.show()


def plot_models_across_strategy(preds_dict, odds, actual, bet_strategy):
    dfs = []
    for key, val in preds_dict.items():
      winnings, bets = bet_strategy(val, odds, actual)
      net = winnings - bets
      x = preds_to_confidences(val)
      dfs.append(pd.DataFrame({key:net},index=x).sort_index().groupby([pd.cut(x, np.linspace(-1,1,21))])[key].sum())
    df = pd.concat(dfs,axis=1).reset_index()
    
    df.plot.bar(x='index',y=preds_dict.keys())
    plt.show()

    df.drop([df.index[0],df.index[-1]]).plot.bar(x='index',y=preds_dict.keys())
    plt.show()

    print(df.sum(axis=0))

In [21]:
if do_manual_bagging:
    bagging_model.fit(X, y.values.ravel(), baggingclassifier__sample_weight=weights.values.ravel())
    preds = bagging_model.predict_proba(X_odds)
    plot_bet_strategies(preds, odds, y_odds)


NameError: name 'bagging_model' is not defined

In [ ]:

if do_sklearn_grid_tree:
    model = grid_search_bag_tree.best_estimator_
    model.fit(X, y.values.ravel(), bagging_clf__sample_weight=weights.values.ravel())
    preds = model.predict_proba(X_odds)
    plot_bet_strategies(preds, odds, y_odds)


In [ ]:

if do_xgboost:
    xgboost_model.fit(X, y.values.ravel(), xgbclassifier__sample_weight=weights.values.ravel())
    preds = xgboost_model.predict_proba(X_odds)
    plot_bet_strategies(preds, odds, y_odds)


In [ ]:

if do_catboost:
    preds = catboost_model.predict_proba(X_odds)
    plot_bet_strategies(preds, odds, y_odds)


In [ ]:
plot_models_across_strategy({'CatBoost':catboost_model.predict_proba(X_odds),
                             'XGBoost':xgboost_model.predict_proba(X_odds),
                             'GridSearch':grid_search_bag_tree.best_estimator_.predict_proba(X_odds),
                             'ManualBagging':bagging_model.predict_proba(X_odds)},
                             odds, y_odds, risky_bet_strategy)

In [ ]:
grid_search_bag_tree.best_estimator_